# Reading DICOM scans an saving as numpy arrays

In [40]:
import os
from pathlib import Path

import numpy as np

In [41]:
from lunglens.data import *

In [42]:
data_dir = Path('../data/extracted')
dest_root_dir = Path('../data/prepared')

## Kaggle: OSIC Pulmonary Fibrosis Progression

In [43]:
ds_dir = data_dir/'osic-pulmonary-fibrosis-progression'
dest_dir = dest_root_dir/'osic-pulmonary-fibrosis-progression/'

In [44]:
all_scans = list(ds_dir.rglob('ID0*'))

In [45]:
convert_dicoms2np(all_scans, dest_dir)

error occured when trying to save scan: ..\data\extracted\osic-pulmonary-fibrosis-progression\train\ID00052637202186188008618
Exception thrown in SimpleITK ImageSeriesReader_Execute: D:\a\1\sitk-build\ITK\Modules\IO\GDCM\src\itkGDCMImageIO.cxx:233:
itk::ERROR: GDCMImageIO(000002AA9CF504B0): Failed to get the buffer!



## COVID19_1110 dataset

In [46]:
ds_dir = data_dir/'COVID19_1110'
dest_dir = dest_root_dir/'COVID19_1110'

In [47]:
all_files = list(ds_dir.rglob('*.nii'))

len(all_files)

1110

In [48]:
convert_dicoms2np(all_files, dest_dir)